# Harnessing Weather Insights for Accurate Energy Load Forecasting

In [4]:
%pip install -r requirements.txt

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.6 MB 7.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/12.6 MB 5.0 MB/s eta 0:00:03
   --------- ------------------------------ 2.9/12.6 MB 4.9 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.6 MB 8.1 MB/s eta 0:00:01
   -------------------------------------- - 12.1/12.6 MB 12.3 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import important libraries

In [55]:
import platform
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression

from pathlib import Path

In [7]:
# Initialize via the full spark path
if platform.system() == 'Windows':
    print("Windows OS detected")
    findspark.init("C:/Spark/spark-3.5.4-bin-hadoop3") # For my local machine
else:
    findspark.init("/usr/local/spark/")

Windows OS detected


In [33]:
# Build the SparkSession
spark = SparkSession.builder \
      .master("local") \
      .appName("Linear Regression Model") \
      .config("spark.executor.memory", "4gb") \
      .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.15.0") \
      .config("spark.executor.timeout", "300s") \
      .config("spark.sql.session.timeZone", "UTC") \
      .getOrCreate()
   
# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.      
sc = spark.sparkContext

### Preprocessing I: Read in Weather and Load Data

In [9]:
# Read in the data

# Folder Structure
# data
# |-- geosphere
# |   |-- YYYY
# |      |-- MM.csv
# |      |-- MM.csv
# |
# |-- transparency
# |   |-- YYYY
# |      |-- MM.xml
# |      |-- MM.xml

# Loop through the geosphere folder and read in the data

# Define the base data folder
base_path = Path("./data/geosphere")

# Collect all data frames first to optimize the union operation
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.csv"):
            print(f"Reading in {month_file}")

            df = spark.read.csv(str(month_file), header=True, inferSchema=True)

            # Convert the time column (string) to a timestamp
            df = df.withColumn("time", to_timestamp(col("time"), "yyyy-MM-dd'T'HH:mmXXX"))
            
            dfs.append(df)
            
            print(f"Read in {df.count()} rows")

if dfs:
    # Combine all DataFrames
    weather = dfs[0]
    for df in dfs[1:]:
        weather = weather.union(df)

    # Aggregate measurements (average from different stations)
    weather = (
        weather.groupBy("time")
        .agg(
            avg("rr").alias("avg_rr"),
            avg("tl").alias("avg_tl"),
            avg("p").alias("avg_p"),
            avg("so_h").alias("avg_so_h"),
            avg("ff").alias("avg_ff"),
        )
        .orderBy("time")
    )

    weather.show(10, truncate=False)
    
    print(weather.count())
    weather.printSchema()
else:
    print("No data found")


Reading in data\geosphere\2022\01.csv
1490
Reading in data\geosphere\2022\02.csv
1346
Reading in data\geosphere\2022\03.csv
1490
Reading in data\geosphere\2022\04.csv
1442
Reading in data\geosphere\2022\05.csv
1490
Reading in data\geosphere\2022\06.csv
1442
Reading in data\geosphere\2022\07.csv
1490
Reading in data\geosphere\2022\08.csv
1490
Reading in data\geosphere\2022\09.csv
1442
Reading in data\geosphere\2022\10.csv
1490
Reading in data\geosphere\2022\11.csv
1442
Reading in data\geosphere\2022\12.csv
1490
Reading in data\geosphere\2023\01.csv
1490
Reading in data\geosphere\2023\02.csv
1346
Reading in data\geosphere\2023\03.csv
1490
Reading in data\geosphere\2023\04.csv
1442
Reading in data\geosphere\2023\05.csv
1490
Reading in data\geosphere\2023\06.csv
1442
Reading in data\geosphere\2023\07.csv
1490
Reading in data\geosphere\2023\08.csv
1490
Reading in data\geosphere\2023\09.csv
1442
Reading in data\geosphere\2023\10.csv
1490
Reading in data\geosphere\2023\11.csv
1442
Reading in 

In [14]:
# Loop through the transparency folder and read in the energy data

# Define base path for transparency data
base_path = Path("./data/transparency")

# Collect DataFrames before performing union (optimization)
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.xml"):
            print(f"Reading transparency data: {month_file}")

            # Read XML data
            df = spark.read.format('xml').option("rowTag", "GL_MarketDocument").load(str(month_file))

            # Extract and explode relevant fields
            df_filtered = df.select(
                col("TimeSeries.Period.timeInterval.start").alias("start_time"),
                col("TimeSeries.Period.timeInterval.end").alias("end_time"),
                col("TimeSeries.Period.resolution").alias("resolution"),
                explode(col("TimeSeries.Period.Point")).alias("Point")  # Flatten Points
            ).select(
                col("start_time"),
                col("end_time"),
                col("resolution"),
                col("Point.position").cast("int").alias("position"),
                col("Point.quantity").cast("double").alias("quantity")
            )

            # Convert ISO 8601 duration (e.g., "PT15M") to minutes dynamically
            df_fixed = df_filtered.withColumn(
                "interval_minutes",
                expr("CAST(SUBSTRING(resolution, 3, LENGTH(resolution) - 3) AS INT)")  # Extracts "15" from "PT15M"
            ).withColumn(
                "actual_time",
                expr("start_time + (position - 1) * interval_minutes * interval 1 minute")
            ).select(
                col("actual_time"),
                col("quantity")
            )
            
            # Aggregate to hourly intervals
            df_hourly = df_fixed.withColumn(
                "hourly_time", date_trunc("hour", col("actual_time"))  # Round down to the hour
            ).groupBy("hourly_time").agg(
                sum("quantity").alias("hourly_quantity")  # Sum all sub-hourly measurements
            )

            # Rename to match the structure of other time series
            df_hourly = df_hourly.select(
                col("hourly_time").alias("timestamp"), col("hourly_quantity").alias("quantity")
            ).orderBy("timestamp")
            
            print(f"Read in {df_hourly.count()} rows")
            
            # Append DataFrame to list
            dfs.append(df_hourly)

# Merge all collected DataFrames
if dfs:
    Load = dfs[0]
    for df in dfs[1:]:
        Load = Load.union(df)

    Load.show(10)
    Load.printSchema()
else:
    print("No data found.")



Reading transparency data: data\transparency\2022\01.xml
Reading transparency data: data\transparency\2022\02.xml
Reading transparency data: data\transparency\2022\03.xml
Reading transparency data: data\transparency\2022\04.xml
Reading transparency data: data\transparency\2022\05.xml
Reading transparency data: data\transparency\2022\06.xml
Reading transparency data: data\transparency\2022\07.xml
Reading transparency data: data\transparency\2022\08.xml
Reading transparency data: data\transparency\2022\09.xml
Reading transparency data: data\transparency\2022\10.xml
Reading transparency data: data\transparency\2022\11.xml
Reading transparency data: data\transparency\2022\12.xml
Reading transparency data: data\transparency\2023\01.xml
Reading transparency data: data\transparency\2023\02.xml
Reading transparency data: data\transparency\2023\03.xml
Reading transparency data: data\transparency\2023\04.xml
Reading transparency data: data\transparency\2023\05.xml
Reading transparency data: data

### Preprocessing II: Combine both Data Frames

In [42]:
if Load is not None and weather is not None:
    # Join the data into a single DataFrame
    data = Load.join(weather, Load.timestamp == weather.time, "inner").drop("time")
    
    # Rename columns for better understanding
    data = data.withColumnRenamed("timestamp", "time")
    data = data.withColumnRenamed("quantity", "load")

    data = data.withColumnRenamed("avg_rr", "rainfall")
    data = data.withColumnRenamed("avg_tl", "temperature")
    data = data.withColumnRenamed("avg_p", "pressure")
    data = data.withColumnRenamed("avg_so_h", "sunshine_duration")
    data = data.withColumnRenamed("avg_ff", "wind_speed")
    
    # Reorder the columns
    data = data.select("load", "time", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed")
    
    # Print the schema and stats
    data.describe().show()
    data.show(10)
    data.printSchema()

+-------+-----------------+-------------------+-----------------+-----------------+------------------+------------------+
|summary|             load|           rainfall|      temperature|         pressure| sunshine_duration|        wind_speed|
+-------+-----------------+-------------------+-----------------+-----------------+------------------+------------------+
|  count|            26304|              26304|            26304|            26304|             26304|             26304|
|   mean|27147.53615419708|0.09834435827250604|10.73671494829684|959.9205348996351|0.2120685066909976|  2.13615229622871|
| stddev|5198.551548542349| 0.6008211250333231|  8.2578055043952|7.875779934448705|0.3395092943434029|1.3306188964835206|
|    min|          16055.0|                0.0|            -11.0|            929.4|               0.0|              0.05|
|    max|          41273.0|               46.3|             33.8|            982.8|               1.0|             10.35|
+-------+---------------

### Machine Learning

In [50]:
# Keep original timestamp before converting to Unix seconds
unix_time_data = data.withColumn("unix_time", unix_timestamp("time"))  # Store Unix time separately

# Extract day of the year (1-365/366) from the original timestamp column
unix_time_data = unix_time_data.withColumn("day_of_year", dayofyear(col("time")))

# Extract time of day in seconds (seconds since midnight)
unix_time_data = unix_time_data.withColumn(
    "time_of_day", expr("hour(time) * 3600 + minute(time) * 60 + second(time)")
)

# Define features for ML model
feature_columns = ["day_of_year", "time_of_day", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transform data
df_ml = assembler.transform(unix_time_data).select("load", "features")

# Show transformed data
df_ml.show(5, truncate=False)



+-------+---------------------------------------------------+
|load   |features                                           |
+-------+---------------------------------------------------+
|22394.0|[1.0,0.0,0.0,13.15,964.5999999999999,0.0,7.65]     |
|21804.0|[1.0,3600.0,0.0,12.75,965.2,0.0,7.05]              |
|20917.0|[1.0,7200.0,0.0,11.3,965.75,0.0,5.3999999999999995]|
|20705.0|[1.0,10800.0,0.0,11.100000000000001,965.75,0.0,3.5]|
|20965.0|[1.0,14400.0,0.0,11.95,965.8,0.0,3.75]             |
+-------+---------------------------------------------------+
only showing top 5 rows



In [51]:
standard_scaler = StandardScaler(inputCol="features", outputCol="features_scaled")

scalar = standard_scaler.fit(df_ml)

scaled_df = scalar.transform(df_ml)

scaled_df = scaled_df.select("load","features_scaled")
scaled_df = scaled_df.withColumn("features",col("features_scaled"))
scaled_df = scaled_df.select("load","features")

scaled_df.show(5, truncate=False)

+-------+----------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                  |
+-------+----------------------------------------------------------------------------------------------------------+
|22394.0|[0.009481837858352278,0.0,0.0,1.5924327586791602,122.47675887702681,0.0,5.749204389188338]                |
|21804.0|[0.009481837858352278,0.1444602776491528,0.0,1.5439937394037484,122.55294180811352,0.0,5.2982863978794486]|
|20917.0|[0.009481837858352278,0.2889205552983056,0.0,1.368402294530381,122.62277616160964,0.0,4.0582619217800024] |
|20705.0|[0.009481837858352278,0.4333808329474585,0.0,1.3441827848926753,122.62277616160964,0.0,2.630354949301854] |
|20965.0|[0.009481837858352278,0.5778411105966113,0.0,1.447115700852925,122.6291247392002,0.0,2.8182374456805577]  |
+-------+-------------------------------------------------------

In [52]:
# Randomly splits this :class:`DataFrame` with the provided weights.
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

train_data.take(5)

[Row(load=20691.0, features=DenseVector([0.019, 0.2889, 0.0, 0.3572, 122.9339, 0.0, 0.4885])),
 Row(load=20715.0, features=DenseVector([0.019, 0.4334, 0.0, 0.3209, 122.8577, 0.0, 0.5636])),
 Row(load=20917.0, features=DenseVector([0.0095, 0.2889, 0.0, 1.3684, 122.6228, 0.0, 4.0583])),
 Row(load=21151.0, features=DenseVector([0.019, 0.1445, 0.0, 0.4481, 122.991, 0.0, 0.263])),
 Row(load=21194.0, features=DenseVector([0.0095, 0.7223, 0.0, 1.4653, 122.5402, 0.0, 3.3819]))]

In [58]:
# Linear regression.
# The learning objective is to minimize the specified loss function, with regularization.
#More about regParam and elasticNetParam (seen as penalties on the loss function that minimises error in predicitons) can be found here: https://runawayhorse001.github.io/LearningApacheSpark/reg.html
# The maxIter parameter sets an upper limit on the number of iterations the optimization algorithm can perform regarding the loss function
# By default the LinearRegression function expects another column named "features"

lr = LinearRegression(labelCol="load", maxIter=10000, regParam=0, elasticNetParam=1)

# Fits a model to the input dataset with optional parameters.
linearModel = lr.fit(train_data)

In [60]:
# Transform  is used to perform value predictions using the trained model
predicted = linearModel.transform(test_data)
predicted.printSchema()

predicted.select("load","features","prediction").show()

root
 |-- load: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-------+--------------------+------------------+
|   load|            features|        prediction|
+-------+--------------------+------------------+
|20705.0|[0.00948183785835...|25297.115386954887|
|20965.0|[0.00948183785835...|  25265.7506699917|
|22671.0|[0.28445513575056...|28727.679859653035|
|22927.0|[0.28445513575056...| 27370.51021741413|
|22966.0|[0.01896367571670...|29107.485141034984|
|23020.0|[0.21808227074210...|27611.210344646133|
|23052.0|[0.04740918929176...| 26650.71034270929|
|23334.0|[0.15170940573363...| 28207.05811014515|
|23533.0|[0.21808227074210...|27775.351404601977|
|23630.0|[0.08533654072517...|29194.864282486822|
|23881.0|[0.08533654072517...|30205.325649301925|
|23998.0|[0.07585470286681...|29416.121492942435|
|24051.0|[0.21808227074210...|  28084.3036134808|
|24135.0|[0.27497329789221...|28389.293179111868|
|24250.0|[0.1517094057336

In [62]:
# regression Summary
trainingSummary = linearModel.summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MSE: %f" % trainingSummary.meanSquaredError)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 4536.848422
r2: 0.242705
MSE: 20582993.608009
MAE: 3819.787288
